# GABAsyn testing

This is a test suite for GABA-ergic synapses on the Gillies-Willshaw STN neuron model, using GABAsyn.mod

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook
# Enable connecting with ipyton console --existing
%connect_info

In [ ]:
# NEURON modules
import neuron
from neuron import h
h.load_file("stdlib.hoc") # Load the standard library
h.load_file("stdrun.hoc") # Load the standard run library

# Add our own modules to Python path
import sys, os.path
scriptdir = os.path.abspath('.')
modulesbase = os.path.normpath(os.path.join(scriptdir, '..'))
sys.path.append(modulesbase)

# Gillies-Willshaw STN model
import gillies_model as gillies

# Plotting & recording
from bgcellmodels.common import analysis

# Physiological parameters
import bgcellmodels.cellpopdata as cpd
from bgcellmodels.cellpopdata import PhysioState, Populations as Pop, NTReceptors as NTR, ParameterSource as Cit

# STN cell testing
from stn_model_evaluation import *

## Configure synapse

First we define a function to set up the inputs. This is an extension function to `stn_model_evaluation.py/STNModelEvaluator`

In [ ]:
def make_GABA_inputs(self, n_gpe_syn):
    """
    Make a single GABAergic synapse on the STN neuron.
    """
    cc = cpd.CellConnector(self.physio_state, self.rng)
        
    # Add GPe inputs using Tsodyks-Markram synapses
    # NOTE: one synapse represents a multi-synaptic contact from one GPe axon
    gpe_syns = []
    gpe_ncs = []
    gpe_stims = []
    gpe_hoc_handlers = []
    gpe_py_handlers = []
    gpe_wvecs = []

    # Pick random segments in dendrites for placing synapses
    is_gpe_target = lambda seg: seg.diam > 1.0 # select proximal dendrites
    dendrites = self.model_data[self.target_model]['sec_refs']['dendrites']
    dend_secrefs = sum(dendrites, [])
    gpe_target_segs = pick_random_segments(dend_secrefs, n_gpe_syn, is_gpe_target, rng=self.rng)

    # Make synapses
    for target_seg in gpe_target_segs:

        # Make poisson spike generator
        stim_rate = 100.0 # hz
        stim_T = stim_rate**-1*1e3
        stimsource = h.NetStim() # Create a NetStim
        stimsource.interval = stim_T # Interval between spikes
        stimsource.number = 8 # max number of spikes
        stimsource.noise = 0.0 # Fractional noise in timing
        gpe_stims.append(stimsource) # Save this NetStim

        # Custom synapse parameters
        syn_mech = 'GABAsyn'
        syn_params = {
            'use_stdp_A': 1,
            'use_stdp_B': 1,
        }

        # Make synapse and NetCon
        syn, nc, wvecs = cc.make_synapse((Pop.GPE, Pop.STN), (stimsource, target_seg), 
                            syn_mech, (NTR.GABAA, NTR.GABAB), 
                            (Cit.Custom, Cit.Chu2015, Cit.Fan2012, Cit.Atherton2013),
                            custom_synpar=syn_params)

        print("Made {} synapse with following parameters:".format(syn_mech))
        for pname in cc.getSynMechParamNames(syn_mech):
            print("{} : {}".format(pname, str(getattr(syn, pname))))

        # Compensate for effect max value Hill factor and U1 on gmax_GABAA and gmax_GABAB
        syn.gmax_GABAA = syn.gmax_GABAA / syn.U1
        syn.gmax_GABAB = syn.gmax_GABAB / 0.21

        # Control netstim
        tstart = 700
        tstop = tstart + 5*stim_T
        stimsource.start = tstart
        turn_off = h.NetCon(None, stimsource)
        turn_off.weight[0] = -1
        def queue_events():
            turn_off.event(tstop)
        
        gpe_py_handlers.append(queue_events)
        gpe_hoc_handlers.append(h.FInitializeHandler(queue_events))

        gpe_ncs.append(nc)
        gpe_ncs.append(turn_off)
        gpe_syns.append(syn)
        gpe_wvecs.extend(wvecs)

    # Save inputs
    new_inputs = {
        'stimweightvec': gpe_wvecs,
        'synapses': gpe_syns,
        'NetCons': gpe_ncs,
        'NetStims': gpe_stims,
        'HocInitHandlers': gpe_hoc_handlers,
        'PyInitHandlers': gpe_py_handlers,
    }
    self.add_inputs('gpe', **new_inputs)

In [ ]:
def make_GLU_inputs(self, n_ctx_syn):
    """
    Make a single Glutamergic synapse on the STN neuron.
    """
    cc = cpd.CellConnector(self.physio_state, self.rng)
        
    # Add CTX inputs using Tsodyks-Markram synapses
    ctx_syns = []
    ctx_ncs = []
    ctx_stims = []
    ctx_hoc_handlers = []
    ctx_py_handlers = []
    ctx_wvecs = []

    # Distribute synapses over dendritic trees
    is_ctx_target = lambda seg: seg.diam <= 1.0			
    dendrites = self.model_data[self.target_model]['sec_refs']['dendrites']
    dend_secrefs = sum(dendrites, [])
    ctx_target_segs = pick_random_segments(dend_secrefs, n_ctx_syn, is_ctx_target, rng=self.rng)

    # Make synapses
    for target_seg in ctx_target_segs:

        # Make poisson spike generator
        stim_rate = 100.0 # hz
        stim_T = stim_rate**-1*1e3
        stimsource = h.NetStim() # Create a NetStim
        stimsource.interval = stim_T # Interval between spikes
        stimsource.number = 5 # max number of spikes
        stimsource.noise = 0.0 # Fractional noise in timing
        ctx_stims.append(stimsource) # Save this NetStim

        # Custom synapse parameters
        syn_mech = 'GLUsyn'
        syn_params = {
            'U1': 0.7,
            'tau_rec': 200., # 1./20. / 2. * 1e3, # 95% recovery of RRP under 20Hz stim (Gradinaru 2009)
            'tau_facil': 1., # no facilitation
        }

        # Make synapse and NetCon
        syn, nc, wvecs = cc.make_synapse((Pop.CTX, Pop.STN), (stimsource, target_seg), 
                            syn_mech, (NTR.AMPA, NTR.NMDA), (Cit.Custom, Cit.Default),
                            custom_synpar=syn_params)

        print("Made {} synapse with following parameters:".format(syn_mech))
        for pname in cc.getSynMechParamNames(syn_mech):
            print("{} : {}".format(pname, str(getattr(syn, pname))))

        # Compensate for effect max value Hill factor and U1 on gmax_GABAA and gmax_GABAB
        syn.gmax_AMPA = syn.gmax_AMPA / syn.U1
        syn.gmax_NMDA = syn.gmax_NMDA / syn.U1

        # Control netstim
        tstart = 850
        tstop = tstart + 10*stim_T
        stimsource.start = tstart
        turn_off = h.NetCon(None, stimsource)
        turn_off.weight[0] = -1
        def queue_events():
            turn_off.event(tstop)
        fih = h.FInitializeHandler(queue_events)
        
        ctx_py_handlers.append(queue_events)
        ctx_hoc_handlers.append(fih)
        ctx_ncs.append(nc)
        ctx_ncs.append(turn_off)
        ctx_syns.append(syn)
        ctx_wvecs.extend(wvecs)

    # Save inputs
    new_inputs = {
        'stimweightvec': ctx_wvecs,
        'synapses': ctx_syns,
        'NetCons': ctx_ncs,
        'NetStims': ctx_stims,
        'PyInitHandlers': ctx_py_handlers,
        'HocInitHandlers': ctx_hoc_handlers,
    }
    self.add_inputs('ctx', **new_inputs)

## Configure recording

Record variables of interest

In [ ]:
def rec_GABA_traces(self, protocol, recordStep=0.025):
    """
    Set up recording Vectors
    """
    somasec = h.SThcell[0].soma
    dendsec = h.SThcell[0].dend1[7]

    # Assign label to each recorded section
    rec_segs = {
        'soma': somasec(0.5), # middle of soma
        'dist_dend': dendsec(0.8), # approximate location along dendrite in fig. 5C
    }

    self.model_data[self.target_model]['rec_segs'][protocol] = rec_segs
    
    # END COMMON PART ##############################################################
    
    # Start trace specification
    traceSpecs = collections.OrderedDict() # for ordered plotting (Order from large to small)
    traceSpecs['t_global'] = {'var':'t'}
    self.rec_dt = recordStep
    
    # Add synapse and segment containing it
    nc = self.model_data[self.target_model]['inputs']['gpe']['NetCons'][0]
    rec_segs['synGABA'] = nc.syn()
    rec_segs['postseg'] = nc.syn().get_segment()

    # Record synaptic variables
    traceSpecs['gA_syn'] = {'pointp':'synGABA', 'var':'g_GABAA'}
    traceSpecs['gB_syn'] = {'pointp':'synGABA', 'var':'g_GABAB'}
    traceSpecs['Rrp_syn'] = {'pointp':'synGABA', 'var':'Rrp'}
    traceSpecs['Use_syn'] = {'pointp':'synGABA', 'var':'Use'}
    traceSpecs['Hill_syn'] = {'pointp':'synGABA', 'var':'G'}

    # Record membrane voltages
    for seclabel, seg in rec_segs.iteritems():
        if isinstance(seg, neuron.nrn.Segment):
            traceSpecs['V_'+seclabel] = {'sec':seclabel, 'loc':seg.x, 'var':'v'}
            
    # Prepare dictionary (label -> Section)
    rec_secs = {}
    for seclabel, hobj in rec_segs.iteritems():
        if isinstance(hobj, neuron.nrn.Segment):
            rec_secs[seclabel] = hobj.sec
        else:
            rec_secs[seclabel] = hobj # point process

    # Use trace specs to make Hoc Vectors
    recData = analysis.recordTraces(rec_secs, traceSpecs, recordStep)

    # Save trace specs and recording Vectors
    self.model_data[self.target_model]['rec_data'][protocol] = {
        'trace_specs': traceSpecs,
        'trace_data': recData,
        'rec_dt': recordStep,
    }

In [ ]:
def rec_GLU_traces(self, protocol, recordStep=0.025):
    """
    Set up recording Vectors
    """
    somasec = h.SThcell[0].soma
    dendsec = h.SThcell[0].dend1[7]

    # Assign label to each recorded section
    rec_segs = {
        'soma': somasec(0.5), # middle of soma
        'dist_dend': dendsec(0.8), # approximate location along dendrite in fig. 5C
    }

    self.model_data[self.target_model]['rec_segs'][protocol] = rec_segs
    
    # END COMMON PART ##############################################################
    
    # Start trace specification
    traceSpecs = collections.OrderedDict() # for ordered plotting (Order from large to small)
    traceSpecs['t_global'] = {'var':'t'}
    self.rec_dt = recordStep
    
    # Add synapse and segment containing it
    nc = self.model_data[self.target_model]['inputs']['ctx']['NetCons'][0]
    rec_segs['synGLU'] = nc.syn()
    rec_segs['postseg'] = nc.syn().get_segment()

    # Record synaptic variables
    traceSpecs['gA_syn'] = {'pointp':'synGLU', 'var':'g_AMPA'}
    traceSpecs['gN_syn'] = {'pointp':'synGLU', 'var':'g_NMDA'}
    traceSpecs['Rrp_syn'] = {'pointp':'synGLU', 'var':'R'}
    traceSpecs['Use_syn'] = {'pointp':'synGLU', 'var':'Use'}

    # Record membrane voltages
    for seclabel, seg in rec_segs.iteritems():
        if isinstance(seg, neuron.nrn.Segment):
            traceSpecs['V_'+seclabel] = {'sec':seclabel, 'loc':seg.x, 'var':'v'}
            
    # Prepare dictionary (label -> Section)
    rec_secs = {}
    for seclabel, hobj in rec_segs.iteritems():
        if isinstance(hobj, neuron.nrn.Segment):
            rec_secs[seclabel] = hobj.sec
        else:
            rec_secs[seclabel] = hobj # point process

    # Use trace specs to make Hoc Vectors
    recData = analysis.recordTraces(rec_secs, traceSpecs, recordStep)

    # Save trace specs and recording Vectors
    self.model_data[self.target_model]['rec_data'][protocol] = {
        'trace_specs': traceSpecs,
        'trace_data': recData,
        'rec_dt': recordStep,
    }

## Configure plotting
Plot variables of interest

In [ ]:
def plot_GABA_traces(self, protocol):
    """
    Plot traces for our simulation
    """
    # Get data
    model = self.target_model
    traceSpecs = self.model_data[model]['rec_data'][protocol]['trace_specs']
    recData = self.model_data[model]['rec_data'][protocol]['trace_data']
    recordStep = self.model_data[model]['rec_data'][protocol]['rec_dt']
    
    # END COMMON PART ##############################################################
    
    # Plot membrane voltages (one figure)
    recV = analysis.match_traces(recData, lambda t: t.startswith('V_'))
    figs_vm = analysis.plotTraces(recV, recordStep, yRange=(-80,40), 
                                    traceSharex=True, oneFigPer='cell')

    # Plot synaptic variables
    syn_traces = analysis.match_traces(recData, lambda t: (not t.startswith('V_')) and t.endswith('syn'))
    n, KD = h.n_GABAsyn, h.KD_GABAsyn # parameters of kinetic scheme
    hillfac = lambda x: x**n/(x**n + KD)
    analysis.plotTraces(syn_traces, recordStep, traceSharex=True, title='Synaptic variables',
                        traceXforms={'Hill_syn': hillfac})

In [ ]:
def plot_GLU_traces(self, protocol):
    """
    Plot traces for our simulation
    """
    # Get data
    model = self.target_model
    traceSpecs = self.model_data[model]['rec_data'][protocol]['trace_specs']
    recData = self.model_data[model]['rec_data'][protocol]['trace_data']
    recordStep = self.model_data[model]['rec_data'][protocol]['rec_dt']
    
    # END COMMON PART ##############################################################
    
    # Plot membrane voltages (one figure)
    recV = analysis.match_traces(recData, lambda t: t.startswith('V_'))
    figs_vm = analysis.plotTraces(recV, recordStep, yRange=(-80,40), 
                                    traceSharex=True, oneFigPer='cell')

    # Plot synaptic variables
    syn_traces = analysis.match_traces(recData, lambda t: (not t.startswith('V_')) and t.endswith('syn'))
    analysis.plotTraces(syn_traces, recordStep, traceSharex=True, title='Synaptic variables')

## Build STN model
Build STN cell and evaluator object

In [ ]:
# Make cell model and evaluator
evaluator = StnModelEvaluator(StnModel.Gillies2005, PhysioState.NORMAL)
evaluator.build_cell(StnModel.Gillies2005)

proto_GABA = StimProtocol.SINGLE_SYN_GABA
proto_GLU = StimProtocol.SINGLE_SYN_GLU

## Simulate single synapse

Now we simulate the cell with this input.

In [ ]:
# Make inputs and recording vectors
make_GABA_inputs(evaluator, n_gpe_syn=1)
rec_GABA_traces(evaluator, proto_GABA)

# Show what we are simulating
h.topology() # not working in notebook

for syn in evaluator.model_data[evaluator.target_model]['inputs']['gpe']['synapses']:
    print("Synapse {} in segment {}".format(syn, syn.get_segment()))

In [ ]:
# Simulate with single synapse
evaluator.init_sim()
evaluator.run_sim()

plot_GABA_traces(evaluator, proto_GABA)

## Simulate multiple synapses

Increase the number of synapses until we get a rebound burst

### Eliciting rebound burst

Rebound burst protocol: (Gillies 2005, Fig. 3-4):

- induced with 500ms -025 nA somatic current injection
- Fig. 3D + 4C-D: hyperpolarized to between -82 ~ -75 mV
- increasing g_CaL or decreasing g_sKCa increased burst duration


Plateau potential protocol: (Gillies 2005, Fig. 10C-D)

- hyperpolarize to -70 mV (below thresold CaT deinactivation, i.e. CaT will be triggered) (see fig. 10C)
- +0.2 nA depolarizing pulse (see fig. 10D) over hyperpolarizing current of duration 50 ms (see fig. 10D, top right)

In [ ]:
# Change SKCa conductance
for sec in h.allsec():
    for seg in sec:
        seg.gk_sKCa = 0.6 * seg.gk_sKCa

# Add some synapses
make_GABA_inputs(evaluator, 1)
make_GLU_inputs(evaluator, 4)

rec_GABA_traces(evaluator, proto_GABA) # uncomment if single synapse simulated
rec_GLU_traces(evaluator, proto_GLU) # uncomment if single synapse simulated

In [ ]:
# Print synapse list
h.topology() # see console from which Jupyter was started
print("List of synapses:")
all_syns = sum((evaluator.model_data[evaluator.target_model]['inputs'][pop]['synapses'] for pop in ['gpe', 'ctx']),[])
for syn in all_syns:
    print("Synapse {} in segment {}".format(syn, syn.get_segment()))

In [ ]:
# Simulate with additional synapses
evaluator.init_sim()
evaluator.run_sim()

In [ ]:
# Plot simulation data
plot_GABA_traces(evaluator, proto_GABA)
plot_GLU_traces(evaluator, proto_GLU)